In [ ]:
! pip install git+https://github.com/huggingface/diffusers
! pip install -U -r requirements.txt
! pip install bitsandbytes
! pip install torch-fidelity
! pip uninstall torch torchvision torchaudio
! pip install torch torchvision torchaudio
! pip install torchmetrics

In [ ]:
! accelerate config default

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from torchvision.transforms import functional as F
from torchmetrics.image.kid import KernelInceptionDistance
from diffusers import DiffusionPipeline
from diffusers.utils import make_image_grid

Training

In [ ]:
! accelerate launch train_dreambooth.py --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" --instance_data_dir="/content/Kamao" --output_dir="/content/DB/" --instance_prompt="a photo of Kamao" --resolution=512 --gradient_checkpointing --use_8bit_adam --train_batch_size=1 --gradient_accumulation_steps=1 --learning_rate=5e-6 --lr_scheduler="constant" --lr_warmup_steps=0 --max_train_steps=500

Image Generation

In [ ]:
model_id = "/content/DB/"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True).to("cuda")

prompt = "a photo of Kamao"
images = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, num_images_per_prompt=16).images
make_image_grid(images[:8], rows=1, cols=8)

Calculating KID

In [ ]:
def preprocess_image(image):
    image = torch.tensor(image).unsqueeze(0)
    image = image.permute(0, 3, 1, 2) / 255.0
    return image

transform = transforms.ToTensor()

dataset_path = "/content/Kamao"
image_paths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path)])
real_images = [np.array(Image.open(path).convert("RGB")) for path in image_paths]
real_images = torch.cat([preprocess_image(image) for image in real_images])

fake_images = images
fake_images = torch.cat([transform(image).unsqueeze(0) for image in fake_images])

In [ ]:
kid = KernelInceptionDistance(normalize=True, subset_size=8)
kid.update(real_images, real=True)
kid.update(fake_images, real=False)

print(f"KID: {kid.compute()}")

Note to grader: due to a technical error with saving, the code for calculating CLIP scores is lost.